In [1]:
from simpletransformers.classification import MultiLabelClassificationModel, MultiLabelClassificationArgs
from simpletransformers.t5 import T5Model, T5Args
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs
from sklearn.utils import shuffle
import pandas as pd
import logging
import os
import csv
import wandb
from sklearn.model_selection import train_test_split

wandb.login()

2023-02-02 14:04:48.788406: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yuanlu. Use `wandb login --relogin` to force relogin


True

In [2]:
folder_name = 'chenyu_51657'

class data:
    def __init__(self, file_name) -> None:

        self.task_type = folder_name.split('_')[0]
        self.file_name = file_name.replace('.csv','')
        self.file_path = folder_name+'/'+self.file_name+'.csv'
        self.type = self.file_name.split('_')[0]

        self.random_state = 96
        self.max_sample = 5000

        if self.type == 'label':

            self.p = float(self.file_name.split('_')[1])
            self.len = int(self.file_name.split('_')[2])

            with open(self.file_path, mode = 'r') as f:
                self.data = pd.DataFrame([[i[0],list([int(r) for r in i[1]])] for i in csv.reader(f)])
                self.data.columns = ['text', 'labels']
            
            self.wandb_job = '{}_{}'.format(self.task_type, self.file_name)
            self.output_dir = 'outputs/'+self.task_type+'/'+self.file_name+'/'

            self.data = shuffle(self.data, random_state=self.random_state)
            self.data.reset_index(inplace=True, drop=True)
            self.data = self.data[:self.max_sample]

            self.train, self.test = train_test_split(self.data, test_size=0.2, random_state=self.random_state)
            del self.data

            self.label_train()
              
        elif self.type == 'mask':

            self.p = 1.0
            self.len = int(self.file_name.split('_')[1])

            with open(self.file_path, mode = 'r') as f:
                self.data = pd.DataFrame([[str(i[0]), str(i[1])] for i in csv.reader(f)]).astype('string')
                self.data.columns = ["input_text", "target_text"]
            
            self.wandb_job = '{}_{}'.format(self.task_type, self.file_name)
            self.output_dir = 'outputs/'+self.task_type+'/'+self.file_name+'/'
            
            self.data = shuffle(self.data, random_state=self.random_state)
            self.data.reset_index(inplace=True, drop=True)
            self.data = self.data[:self.max_sample]

            self.train, self.test = train_test_split(self.data, test_size=0.2, random_state=self.random_state)
            self.train, self.eval = train_test_split(self.train, test_size=0.2, random_state=self.random_state)
            del self.data

            self.mask_train()
    

    def label_train(self):
        # Optional model configuration
        model_args = MultiLabelClassificationArgs(num_train_epochs=2)
        model_args.wandb_project = self.wandb_job
        model_args.output_dir = self.output_dir


        # Create a MultiLabelClassificationModel
        model = MultiLabelClassificationModel(
            "bert",
            "bert-base-chinese",
            num_labels=self.len,
            args=model_args
        )

        # Train the model
        model.train_model(self.train)
        self.model = model

        # Evaluate the model
        result, model_outputs, wrong_predictions = model.eval_model(
            self.test
        )

    
    def mask_train(self):
        # Configure the model
        model_args = Seq2SeqArgs()
        model_args.num_train_epochs = 2
        model_args.evaluate_generated_text = True
        model_args.output_dir = self.output_dir
        model_args.wandb_project = self.wandb_job
        model_args.evaluate_during_training = True
        model_args.evaluate_during_training_verbose = True

        model = Seq2SeqModel(
            'bert',
            'bert-base-chinese',
            'bert-base-chinese',
            args=model_args,
        )

        # Train the model
        model.train_model(self.train, eval_data = self.eval)
        self.model = model

        # Evaluate the model
        result = self.model.eval_model(self.test)



        

mask = data('mask_64.csv')
mask.model.predict(mask.test)